In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/09 15:46:10 WARN Utils: Your hostname, Soroushs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.101 instead (on interface en0)
25/11/09 15:46:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/09 15:46:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df = spark.read.csv('datasets/phone.csv', header=True, inferSchema=True)
df

DataFrame[brand: string, model: string, price_usd: int, ram_gb: int, storage_gb: int, camera_mp: int, battery_mah: int, display_size_inch: double, charging_watt: int, 5g_support: string, os: string, processor: string, rating: double, release_month: string, year: int]

In [8]:
df.show(n=10)

+-------+--------------------+---------+------+----------+---------+-----------+-----------------+-------------+----------+-------+------------------+------+-------------+----+
|  brand|               model|price_usd|ram_gb|storage_gb|camera_mp|battery_mah|display_size_inch|charging_watt|5g_support|     os|         processor|rating|release_month|year|
+-------+--------------------+---------+------+----------+---------+-----------+-----------------+-------------+----------+-------+------------------+------+-------------+----+
|   Oppo|             A98 111|      855|    16|       128|      108|       6000|              6.6|           33|       Yes|Android|         Helio G99|   3.8|     February|2025|
| Realme|         11 Pro+ 843|      618|     6|       128|       64|       4500|              6.9|          100|       Yes|Android|         Tensor G4|   4.4|       August|2025|
| Xiaomi|Redmi Note 14 Pro...|      258|    16|        64|       64|       4000|              6.8|           44|   

In [9]:
df.printSchema()

root
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- price_usd: integer (nullable = true)
 |-- ram_gb: integer (nullable = true)
 |-- storage_gb: integer (nullable = true)
 |-- camera_mp: integer (nullable = true)
 |-- battery_mah: integer (nullable = true)
 |-- display_size_inch: double (nullable = true)
 |-- charging_watt: integer (nullable = true)
 |-- 5g_support: string (nullable = true)
 |-- os: string (nullable = true)
 |-- processor: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- release_month: string (nullable = true)
 |-- year: integer (nullable = true)



In [11]:
df = df.drop(
    'ram_gb', 'storage_gb', 'camera_mp', 'battery_mah',
    'display_size_inch', 'charging_watt', '5g_support',
    'processor', 'release_month'
)
df.show(10)

+-------+--------------------+---------+-------+------+----+
|  brand|               model|price_usd|     os|rating|year|
+-------+--------------------+---------+-------+------+----+
|   Oppo|             A98 111|      855|Android|   3.8|2025|
| Realme|         11 Pro+ 843|      618|Android|   4.4|2025|
| Xiaomi|Redmi Note 14 Pro...|      258|Android|   4.1|2025|
|   Vivo|            V29e 744|      837|Android|   4.1|2025|
|  Apple|iPhone 16 Pro Max...|      335|    iOS|   3.5|2025|
|OnePlus|          Nord 4 295|      938|Android|   3.7|2025|
|Infinix|       GT 20 Pro 112|      418|Android|   4.1|2025|
|  Apple|iPhone 16 Pro Max...|     1447|    iOS|   4.7|2025|
|Infinix|          Hot 40 111|     1354|Android|   4.0|2025|
| Realme|        GT 7 Pro 668|      311|Android|   4.4|2025|
+-------+--------------------+---------+-------+------+----+
only showing top 10 rows


In [15]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['price_usd'],
    outputCols=['imputed_price'],
).setStrategy("median")

imputer.fit(df).transform(df).show()

+-------+--------------------+---------+-------+------+----+-------------+
|  brand|               model|price_usd|     os|rating|year|imputed_price|
+-------+--------------------+---------+-------+------+----+-------------+
|   Oppo|             A98 111|      855|Android|   3.8|2025|          855|
| Realme|         11 Pro+ 843|      618|Android|   4.4|2025|          618|
| Xiaomi|Redmi Note 14 Pro...|      258|Android|   4.1|2025|          258|
|   Vivo|            V29e 744|      837|Android|   4.1|2025|          837|
|  Apple|iPhone 16 Pro Max...|      335|    iOS|   3.5|2025|          335|
|OnePlus|          Nord 4 295|      938|Android|   3.7|2025|          938|
|Infinix|       GT 20 Pro 112|      418|Android|   4.1|2025|          418|
|  Apple|iPhone 16 Pro Max...|     1447|    iOS|   4.7|2025|         1447|
|Infinix|          Hot 40 111|     1354|Android|   4.0|2025|         1354|
| Realme|        GT 7 Pro 668|      311|Android|   4.4|2025|          311|
|Samsung|  Galaxy Z Fold6

In [18]:
df.count()

1000

In [23]:
df.groupBy('brand').avg('price_usd').show()


+-------+-----------------+
|  brand|   avg(price_usd)|
+-------+-----------------+
|Infinix|839.1714285714286|
| Realme|771.7809523809524|
| Xiaomi|827.7368421052631|
|   Oppo|826.3272727272728|
|Samsung|791.7238095238096|
| Google|808.0603448275862|
|OnePlus|           812.25|
|   Vivo|807.9016393442623|
|  Apple|835.6915887850467|
+-------+-----------------+

